In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import pickle

In [2]:
url = 'https://www.macrotrends.net/states/louisiana/population'
r = requests.get(url)
r

<Response [200]>

In [3]:
page = r.text

In [4]:
soup = BeautifulSoup(page)

In [5]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="https://www.macrotrends.net/states/louisiana/population" rel="canonical"/>
<title>Louisiana Population 1900-2020 | MacroTrends</title>
<meta content="Chart and table of population level and growth rate for the state of Louisiana from 1900 to 2020." name="description"/>
<meta content="" name="robots"/>
<link href="/assets/images/icons/FAVICON/macro-trends_favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="1228954C688F5907894001CD8E5E624B" name="msvalidate.01"/>
<meta content="6MnD_3iDtAP1ZyoGK1YMyVIVck4r5Ws80I9xD3ue4_A" name="google-site-verification"/>

In [6]:
soup = soup.find('div', class_='col-xs-6')

In [7]:
a_list = []
for column in soup.find_all('td'):
    a_list.append(column.text)

In [8]:
year = a_list[0::3]

In [9]:
population = a_list[1::3]

In [10]:
year_population = dict(zip(year, population))

In [11]:
state_pop = pd.DataFrame.from_dict(year_population, orient = 'index', columns=["population"])

In [12]:
state_pop.reset_index(inplace = True)
state_pop.rename(columns={"index":"dtime"}, inplace = True)

In [13]:
state_pop.replace(',','', regex = True, inplace = True)
state_pop['dtime'] = pd.to_datetime(state_pop['dtime']).dt.to_period('M')
state_pop

,dtime,population
0,2020-01,4645318
1,2019-01,4658285
2,2018-01,4664450
3,2017-01,4673673
4,2016-01,4681346
...,...,...
116,1904-01,1495000
117,1903-01,1467000
118,1902-01,1440000
119,1901-01,1412000


In [14]:
state_pop = state_pop.astype({'population':'int32'})

In [15]:
#target features - sales - is disaggregated by month
#steps below performed to disaggregate population from year to month
state_pop = state_pop.set_index('dtime').resample('M').asfreq()
state_pop.reset_index(inplace = True)
state_pop.tail(30)

,dtime,population
1411,2017-08,NaN
1412,2017-09,NaN
1413,2017-10,NaN
1414,2017-11,NaN
1415,2017-12,NaN
1416,2018-01,4664450.0
1417,2018-02,NaN
1418,2018-03,NaN
1419,2018-04,NaN
1420,2018-05,NaN


In [16]:
state_pop.fillna(method='ffill', inplace=True)
state_pop.tail(30)

,dtime,population
1411,2017-08,4673673.0
1412,2017-09,4673673.0
1413,2017-10,4673673.0
1414,2017-11,4673673.0
1415,2017-12,4673673.0
1416,2018-01,4664450.0
1417,2018-02,4664450.0
1418,2018-03,4664450.0
1419,2018-04,4664450.0
1420,2018-05,4664450.0


In [17]:
state_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1441 entries, 0 to 1440
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype    
---  ------      --------------  -----    
 0   dtime       1441 non-null   period[M]
 1   population  1441 non-null   float64  
dtypes: float64(1), period[M](1)
memory usage: 22.6 KB


In [18]:
state_pop['dtime'] = state_pop['dtime'].astype('str')
state_pop['dtime'] = pd.to_datetime(state_pop['dtime'])
state_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1441 entries, 0 to 1440
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   dtime       1441 non-null   datetime64[ns]
 1   population  1441 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 22.6 KB


In [19]:
state_pop.to_pickle('state_pop.pkl')